In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings; warnings.filterwarnings("ignore")

In [2]:
user_df = pd.read_csv("/tmp/rating.csv")

user_df.isnull().sum().sum()

0

In [3]:
user_df.iloc[:, 0].nunique()

610

In [4]:
user_df.iloc[:, 1].nunique()

9724

In [5]:
user_df.loc[:, "rating"].min()

0.5

In [6]:
user_df.loc[:, "rating"].max()

5.0